In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv('/Users/allisongross/repos/titanic/pyconuk-introtutorial/data/train.csv')

In [2]:
df=df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 62.7+ KB


In [4]:
age_mean = df['Age'].mean()
df['Age']=df['Age'].fillna(age_mean)

In [5]:
from scipy.stats import mode

mode_emb = mode(df['Embarked'])[0][0]
df['Embarked']=df['Embarked'].fillna(mode_emb)

/Users/allisongross/anaconda2/anaconda/lib/python2.7/site-packages/scipy/stats/stats.py:257: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/Users/allisongross/anaconda2/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [6]:
from sklearn import preprocessing

le_sex = preprocessing.LabelEncoder()
le_sex.fit(df['Sex'])
df['Gender']=le_sex.transform(df['Sex'].values)

In [7]:
le_emb=preprocessing.LabelEncoder()
le_emb.fit(df['Embarked'])
df['Port']=le_emb.transform(df['Embarked'].values)

In [8]:
df=df.drop(['Embarked', 'Sex'], axis=1)

cols= df.columns.tolist()
print cols

['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Gender', 'Port']


In [9]:
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived       891 non-null int64
PassengerId    891 non-null int64
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Gender         891 non-null int64
Port           891 non-null int64
dtypes: float64(2), int64(7)
memory usage: 62.7 KB


In [10]:
train_data=df.values

In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 100)
model= model.fit(train_data[0:,2:], train_data[0:,0])

In [12]:
df_test=pd.read_csv('./pyconuk-introtutorial/data/test.csv')
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [13]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           417 non-null float64
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 26.2+ KB


In [14]:
df_test['Age']=df_test['Age'].fillna(age_mean)

fare_means= df.pivot_table('Fare', index='Pclass', aggfunc='mean')

fare_means

Pclass
1    84.154687
2    20.662183
3    13.675550
Name: Fare, dtype: float64

In [15]:
df_test['Fare']=df_test[['Fare', 'Pclass']].apply(lambda x:
                                               fare_means[x['Pclass']] if pd.isnull(x['Fare'])
                                               else x['Fare'], axis=1)

In [16]:
df_test['Gender']=le_sex.transform(df_test['Sex'].values)
df_test['Port']=le_emb.transform(df_test['Embarked'].values)

df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

test_data = df_test.values

output = model.predict(test_data[:,1:])

result = np.c_[test_data[:,0].astype(int), output.astype(int)]
df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])

In [17]:
df_result.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1


In [20]:
%pwd

u'/Users/allisongross/repos/titanic'

In [21]:
df_result.to_csv('./Results/titanic_ex1_1.csv')